In [ ]:
!pip install fuzzywuzzy
!pip install rapidfuzz
!pip install nltk
!pip install difflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement difflib (from versions: none)
ERROR: No matching distribution found for difflib


In [ ]:
import pandas as pd

# 1) Import pustaka yang dibutuhkan
from fuzzywuzzy import fuzz
from rapidfuzz import fuzz as rapidfuzz_module
import nltk
from nltk.metrics import edit_distance
from difflib import SequenceMatcher

# Jika belum pernah, jalankan sekali untuk download data NLTK (untuk edit_distance tidak perlu)
# nltk.download('punkt')

In [ ]:
# 2) Baca dataset
df = pd.read_csv('PoPCites.csv')
authors = df['Authors'].astype(str).tolist()

In [ ]:
authors

['I Lewaa, MS Hafez, MA Ismail',
 'M El Abassi, M Amnai, A Choukri, Y Fakhri…',
 'J Yang, K Xian, P Wang, Y Zhang',
 'J Yang, S Quan, P Wang…',
 'Y Zhu, J Yang',
 'LL Sharabi',
 'DA Wood',
 'B Sun, T Alkhalifah',
 'T Lei, Z Lei',
 'J Yang, C Zhao, K Xian, A Zhu, Z Cao',
 'DA Wood',
 'DA Wood',
 'T Hayashi, Y Ohsawa',
 'Z Du, Y Zuo, J Qiu, X Li, Y Li, H Guo, X Hong…',
 'B Engquist, K Ren, Y Yang',
 'A Alali, V Kazei, B Sun, T Alkhalifah',
 'S Bader, X Wu, S Fomel',
 'C Wüstenhagen, K John, S Langner, M Brede…',
 'N Ahmadi, H Sand, P Papotti',
 'W Xue, D Vatsalan, W Hu…',
 'SJ Grannis, H Xu, JR Vest…',
 'DA Wood',
 'N Wang, A Vlachokostas, M Borkum, H Bergmann…',
 'P Verschuuren, S Palazzo, T Powell, S Sutton…',
 "R D'Alberto, M Raggi",
 'H Liu, ZW Hu, Z Yang, M Wang',
 'X Gao, J Yi, L Liu, L Tan',
 'K Blythe',
 'X Fu, Y Zhang, W Zhang, Q Li, T Kong',
 'J Dai, Y Teng, Z Zhang, Z Yu, G Sheng, X Jiang',
 'X Peng, H Liu, K Siggers, Z Liu',
 'NE Kired, F Ravat, J Song, O Teste',
 'B Aksoy, S

In [ ]:
# 3) Siapkan list untuk menampung hasil
results = []

In [ ]:
# 4) Looping pairwise
for i in range(len(authors)):
    for j in range(i + 1, len(authors)):
        s1 = authors[i]
        s2 = authors[j]

        # FuzzyWuzzy (Levenshtein → ratio 0–100)
        score_fw = fuzz.ratio(s1, s2)

        # RapidFuzz (Levenshtein → ratio 0–100)
        score_rf = rapidfuzz_module.ratio(s1, s2)

        # NLTK (jarak edit → konversi ke % kemiripan)
        dist = edit_distance(s1, s2)
        max_len = max(len(s1), len(s2))
        score_nltk = (1 - dist / max_len) * 100 if max_len > 0 else 0

        # Difflib (SequenceMatcher → ratio 0.0–1.0)
        score_diff = SequenceMatcher(None, s1, s2).ratio() * 100

        results.append({
            'idx_1': i,
            'idx_2': j,
            'Authors_1': s1,
            'Authors_2': s2,
            'FuzzyWuzzy_%': round(score_fw, 2),
            'RapidFuzz_%': round(score_rf, 2),
            'NLTK_%': round(score_nltk, 2),
            'DiffLib_%': round(score_diff, 2)
        })

In [ ]:
len(results)

4950

In [ ]:
results[1599]

{'idx_1': 17,
 'idx_2': 70,
 'Authors_1': 'C Wüstenhagen, K John, S Langner, M Brede…',
 'Authors_2': 'L Zhao, Y Liu, Y Lu, Y Sun, J Li, K Yao',
 'FuzzyWuzzy_%': 22,
 'RapidFuzz_%': 34.57,
 'NLTK_%': 21.43,
 'DiffLib_%': 22.22}

In [ ]:
# prompt: convert "results" variable to dataframe and download it as excel

# Convert the list of dictionaries to a pandas DataFrame
df_results = pd.DataFrame(results)

# Save the DataFrame to an Excel file
df_results.to_excel('author_similarity_results.xlsx', index=False)

print("Results DataFrame converted to Excel and saved as 'author_similarity_results.xlsx'")

Results DataFrame converted to Excel and saved as 'author_similarity_results.xlsx'
